# Setup

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import os

import pandas as pd
import altair as vg
import seaborn as sns

import numpy as np
import math as m

from itertools import combinations

# Data Prep

In [3]:
filepath = '../data/demand_raw/Dados_v3.xlsx'
df = pd.read_excel(filepath)

In [4]:
df.head()

,Sigla,Nome,Latitude,Longitude,Município,Modal,Rio,Latitude_2,Longitude_2,OBS,Considerar,Carga_2023,Carga_2024,GN,GNL,Conteineres/Ano
0,AM-000,MANAUS,-3.138087,-60.027484,Manaus,NaN,Negro,-3.138087,-60.027484,Porto de Manaus,0,0,0,NaN,NaN,NaN
1,AM-083,SACAMBU,-3.272442,-60.952701,Manacapuru,Hidroviário,Solimões,-3.274633,-60.933956,"Lago Cabaliana, UTE Sacambu; este nó será agre...",0,1487,1511,4.558757e+05,759.792829,15.195857
2,AM-028,CAMPINAS,-3.278834,-61.099313,Manacapuru,Hidroviário,Solimões,-3.278834,-61.099313,Lago Cabaliana + Rio Paraná do Aramã; este nó ...,1,1085,1099,3.315734e+05,552.622316,11.052446
3,AM-023,CAAPIRANGA,-3.324601,-61.212903,Manacapuru,Gasoduto,Solimões,-3.278834,-61.099313,Verificar acesso ao município; supondo entrega...,0,9984,10253,3.093378e+06,5155.629306,103.112586
4,AM-004,ANAMÃ,-3.572283,-61.407035,Anamã,Gasoduto,Solimões,-3.572283,-61.407035,NaN,0,12675,13347,4.026852e+06,6711.419520,134.228390


In [5]:
df.dtypes

Sigla               object
Nome                object
Latitude           float64
Longitude          float64
Município           object
Modal               object
Rio                 object
Latitude_2         float64
Longitude_2        float64
OBS                 object
Considerar           int64
Carga_2023           int64
Carga_2024           int64
GN                 float64
GNL                float64
Conteineres/Ano    float64
dtype: object

# EDA

In [6]:
df.describe()

,Latitude,Longitude,Latitude_2,Longitude_2,Considerar,Carga_2023,Carga_2024,GN,GNL,Conteineres/Ano
count,86.000000,86.000000,86.000000,86.000000,96.000000,96.000000,96.000000,8.200000e+01,82.000000,82.000000
mean,-3.773587,-63.488361,-3.778263,-63.564175,0.625000,20103.364583,20673.541667,6.879314e+06,11465.524114,229.310482
std,2.076320,4.406264,2.087978,4.343704,0.486664,31821.860685,32843.231021,1.043984e+07,17399.725018,347.994500
min,-8.747551,-72.581236,-8.747551,-72.581236,0.000000,0.000000,0.000000,6.516820e+04,108.613667,2.172273
25%,-4.721653,-67.061608,-4.719700,-67.061608,0.000000,1568.250000,1626.750000,7.980842e+05,1330.140293,26.602806
50%,-3.354899,-62.973198,-3.350754,-63.020109,1.000000,10170.000000,10398.000000,3.806758e+06,6344.596973,126.891939
75%,-2.746235,-59.750366,-2.707575,-60.004683,1.000000,21500.500000,22015.500000,7.134032e+06,11890.053804,237.801076
max,1.190652,-56.668167,1.190652,-56.668167,1.000000,210942.000000,220434.000000,6.650596e+07,110843.264444,2216.865289


In [7]:
df.Rio.value_counts(dropna=False)

Solimões    23
Negro       11
Madeira     11
Amazonas    11
NaN         10
Purus        9
--           9
Juruá        8
Japurá       3
Javari       1
Name: Rio, dtype: int64

In [8]:
# Drop localities without rivers attached
df = df[~df.Rio.isin(['--'])]
df = df.dropna(subset=['Rio'])

## Mapa de demanda

In [9]:
# Topo map
filepath = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-13-mun.json'

topo = vg.Chart(filepath).mark_geoshape(
    stroke="#dd",
    strokeWidth=2.5,
    color="lightgray",
).project(
    type='mercator'
)

# Demanda
selection = vg.selection_multi(fields=['Rio'], bind='legend')
points = vg.Chart(df).mark_point().encode(
    vg.Longitude('Longitude:Q',
                ),
    vg.Latitude('Latitude:Q',
               ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='set1')),
                        vg.value('lightgray'),
                       ),
    size=vg.Size('Conteineres/Ano:Q', 
                 scale=None,
                 legend=vg.Legend(title='Conteineres por Ano')),
    tooltip=['Sigla', 'Nome', 'Latitude', 'Longitude', 'Município', 'Modal', 'Rio',
             'Latitude_2', 'Longitude_2', 'OBS', 'Considerar', 'Carga_2023',
             'Carga_2024', 'GN', 'GNL', 'Conteineres/Ano']
)

# Mapa + Demanda + Configs
vg.layer(
    topo,
    points,
).properties(
    width=600,
    height=600,
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=14,
    titleFontSize=20,
).configure_legend(
    titleFontSize=16,
    labelFontSize=14, 
).add_selection(
    selection
)

alt.LayerChart(...)

## Demanda (permite zoom e seleção na legenda)

In [10]:
selection = vg.selection_multi(fields=['Rio'], bind='legend')

chart = vg.Chart(df).mark_point().encode(
    vg.X('Longitude:Q',
          scale=vg.Scale(zero=False)
        ),
    vg.Y('Latitude:Q',
         scale=vg.Scale(zero=False)
        ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='set1')),
                        vg.value('lightgray'),
                       ),
    size=vg.Size('Conteineres/Ano:Q', 
                 scale=None,
                 legend=vg.Legend(title='Conteineres por Ano')),
    tooltip=['Sigla', 'Nome', 'Latitude', 'Longitude', 'Município', 'Modal', 'Rio',
             'Latitude_2', 'Longitude_2', 'OBS', 'Considerar', 'Carga_2023',
             'Carga_2024', 'GN', 'GNL', 'Conteineres/Ano']
).properties(
    width=600,
    height=600,
).configure_title(
    fontSize=24,
).configure_axis(
    labelFontSize=14,
    titleFontSize=20,
).configure_legend(
    titleFontSize=16,
    labelFontSize=14, 
).add_selection(
    selection
).interactive()

chart

alt.Chart(...)

## Pareto

In [11]:
df_pareto = df.copy()
df_pareto = df_pareto[df_pareto['Conteineres/Ano'].notna()]
df_pareto = df_pareto.sort_values(by=['Conteineres/Ano'], ascending=False)

df_pareto["count cumsum"] = df_pareto['Conteineres/Ano'].cumsum()
df_pareto["cumpercentage"] = df_pareto["count cumsum"]/(df_pareto['Conteineres/Ano'].sum())


sort_order = df_pareto["Nome"].tolist()

selection = vg.selection_multi(fields=['Rio'], bind='legend')

# Create Base
base = vg.Chart(df_pareto).encode(
    vg.X("Nome:O",
         sort=sort_order),
).properties(
    width=800,
    height=600,
)
# Bars chart
bars = base.mark_bar().encode(
    vg.Y("Conteineres/Ano:Q",
    axis=vg.Axis(labelFontSize=15)
        ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='set1')),
                        vg.value('lightgray'),
                       ),
).properties(
    width=800,
    height=600,
)
# Line chart
line = base.mark_line(strokeWidth=1.5, color="#cb4154" ).encode(
    vg.Y('cumpercentage:Q',
    title='Cumulative Count',
    axis=vg.Axis(format=".0%", labelFontSize=15, )
        ),
    text = vg.Text('cumpercentage:Q')
)
# Mark the percentage values on the line with Circle marks
points = base.mark_circle(strokeWidth= 3, color = "#cb4154").encode(
    vg.Y('cumpercentage:Q', axis=None),
    tooltip=['Nome', 'Município', 'Modal', 'Rio', 'OBS', 'Considerar', 'Conteineres/Ano'],
)

# Layer all the elements together 
(bars + line + points).resolve_scale(
    y = 'independent'
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=10,
    titleFontSize=20,
).add_selection(
    selection
).interactive()

alt.LayerChart(...)

# Considerar

In [12]:
df.loc[df.Nome == 'ALTEROSA', 'Considerar'] = 0

In [13]:
# Topo map
filepath = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-13-mun.json'

topo = vg.Chart(filepath).mark_geoshape(
    stroke="#dd",
    strokeWidth=0.3,
    opacity=0.2,
    color="lightgray",
).project(
    type='mercator'
)

# Demanda
selection = vg.selection_multi(fields=['Rio'], bind='legend')
points = vg.Chart(df[(df.Considerar == 1) & (df.Nome != 'ALTEROSA') & (df.Rio != 'Japurá')]
                 ).mark_line(point=True, size=3).encode(
    vg.Longitude('Longitude:Q',
                ),
    vg.Latitude('Latitude:Q',
               ),
    color= vg.condition(selection,
                        vg.Color('Rio:N', scale=vg.Scale(scheme='set1')),
                        vg.value('lightgray'),
                       ),
    #size=vg.Size('Conteineres/Ano:Q', 
    #             scale=None,
    #             legend=vg.Legend(title='Conteineres por Ano')),
    tooltip=['Sigla', 'Nome', 'Latitude', 'Longitude', 'Município', 'Modal', 'Rio',
             'Latitude_2', 'Longitude_2', 'OBS', 'Considerar', 'Carga_2023',
             'Carga_2024', 'GN', 'GNL', 'Conteineres/Ano']
)

# Mapa + Demanda + Configs
vg.layer(
    topo,
    points,
).properties(
    width=600,
    height=600,
).configure_title(
    fontSize=24,
).configure_axis(
    grid=False,
    labelFontSize=14,
    titleFontSize=20,
).configure_legend(
    titleFontSize=16,
    labelFontSize=14, 
).add_selection(
    selection
)

alt.LayerChart(...)

A fazer:

[DONE] 1) Levantar a matriz de distâncias (Solimões, Madeira e Amazonas) para qualquer origem e destino

2) Calcular tempo de ciclo:

Tempo de ciclo = Tempo de Nav + Tempo de mov de carga

Tempo de Nav = distancia / vel. média

Tempo de Mov de carga = 4 x 5 min

(Demanda por ciclo de 14 dias, 14 dias, 7 dias) - dos nós extremos baseado na velocidade média (ajustado pra cima, múltiplos de 7 são convenientes)

- Tempo de Porto total é de 4 movimentos do conteiner (carreta)

- Estabelecer a frequência de entrega (1, 2, 3 vezes por ciclo)

- Demanda por entrega - aredondar por cima (= Demanda / frequencia)

3) Levantamento da capacidade de navios

4) Geração de rotas (itertools -Python) 
 
 - Limitar trechos entre Portos com alta distância (300 km - verificar qual limite)
 - Capacidade máxima do navio

# Distâncias

In [14]:
filepath = '../data/distances/distance.xlsx'
df_dist = pd.read_excel(filepath, index_col=0)

In [15]:
df_dist.style.format("{:.2f}").background_gradient('Blues')

,BENJAMIN CONSTANT,TABATINGA,FEIJOAL,BELÉM DO SOLIMÕES,SANTA RITA DO WEILL,SÃO PAULO DE OLIVENÇA,AMATURÁ,SANTO ANTÔNIO DO IÇÁ,TONANTINS,JUTAÍ,FONTE BOA,UARINI,ALVARÃES,TEFÉ,CAIAMBÉ,CAMPINAS,NOVO REMANÇO,VILA AMAZÔNIA,PARINTINS,URUCARÁ,AUGUSTO MONTENEGRO,URUCURITUBA,ITACOATIARA,AUTAZES,NOVA OLINDA DO NORTE,AXINIM,BORBA,NOVO ARIPUANÃ,MANICORÉ,AUXILIADORA,HUMAITÁ,MANAUS
BENJAMIN CONSTANT,0.00,12.25,53.85,98.75,161.80,212.65,299.80,353.40,385.30,524.70,636.75,802.95,845.05,873.30,903.80,1449.90,1577.35,1913.35,1905.85,1854.35,1796.45,1687.85,1650.25,1665.15,1692.05,1729.05,1780.15,1926.15,2073.15,2178.15,2428.15,2045.90
TABATINGA,12.25,0.00,51.75,96.65,159.70,210.55,297.70,351.30,383.20,522.60,634.65,800.85,842.95,871.20,901.70,1447.80,1575.25,1911.25,1903.75,1852.25,1794.35,1685.75,1648.15,1663.05,1689.95,1726.95,1778.05,1924.05,2071.05,2176.05,2426.05,2043.80
FEIJOAL,53.85,51.75,0.00,45.00,108.05,158.90,246.05,299.65,331.55,470.95,583.00,749.20,791.30,819.55,850.05,1396.15,1523.60,1859.60,1852.10,1800.60,1742.70,1634.10,1596.50,1611.40,1638.30,1675.30,1726.40,1872.40,2019.40,2124.40,2374.40,1992.15
BELÉM DO SOLIMÕES,98.75,96.65,45.00,0.00,63.55,114.40,201.55,255.15,287.05,426.45,538.50,704.70,746.80,775.05,805.55,1351.65,1479.10,1815.10,1807.60,1756.10,1698.20,1589.60,1552.00,1566.90,1593.80,1630.80,1681.90,1827.90,1974.90,2079.90,2329.90,1947.65
SANTA RITA DO WEILL,161.80,159.70,108.05,63.55,0.00,50.90,138.05,191.65,223.55,362.95,475.00,641.20,683.30,711.55,742.05,1288.15,1415.60,1751.60,1744.10,1692.60,1634.70,1526.10,1488.50,1503.40,1530.30,1567.30,1618.40,1764.40,1911.40,2016.40,2266.40,1884.15
SÃO PAULO DE OLIVENÇA,212.65,210.55,158.90,114.40,50.90,0.00,87.35,140.95,172.85,312.25,424.30,590.50,632.60,660.85,691.35,1237.45,1364.90,1700.90,1693.40,1641.90,1584.00,1475.40,1437.80,1452.70,1479.60,1516.60,1567.70,1713.70,1860.70,1965.70,2215.70,1833.45
AMATURÁ,299.80,297.70,246.05,201.55,138.05,87.35,0.00,54.10,86.00,225.40,337.45,503.65,545.75,574.00,604.50,1150.60,1278.05,1614.05,1606.55,1555.05,1497.15,1388.55,1350.95,1365.85,1392.75,1429.75,1480.85,1626.85,1773.85,1878.85,2128.85,1746.60
SANTO ANTÔNIO DO IÇÁ,353.40,351.30,299.65,255.15,191.65,140.95,54.10,0.00,34.10,173.50,285.55,451.75,493.85,522.10,552.60,1098.70,1226.15,1562.15,1554.65,1503.15,1445.25,1336.65,1299.05,1313.95,1340.85,1377.85,1428.95,1574.95,1721.95,1826.95,2076.95,1694.70
TONANTINS,385.30,383.20,331.55,287.05,223.55,172.85,86.00,34.10,0.00,139.90,251.95,418.15,460.25,488.50,519.00,1065.10,1192.55,1528.55,1521.05,1469.55,1411.65,1303.05,1265.45,1280.35,1307.25,1344.25,1395.35,1541.35,1688.35,1793.35,2043.35,1661.10
JUTAÍ,524.70,522.60,470.95,426.45,362.95,312.25,225.40,173.50,139.90,0.00,112.10,278.30,320.40,348.65,379.15,925.25,1052.70,1388.70,1381.20,1329.70,1271.80,1163.20,1125.60,1140.50,1167.40,1204.40,1255.50,1401.50,1548.50,1653.50,1903.50,1521.25


# Definição do horizonte de planejamento

In [16]:
df['Rio'] = df['Rio'].apply(lambda x: 'Solimoes' if x == 'Solimões' else x)

In [17]:
# Dataframe com apenas localidades consideradas e rios
rios = ['Amazonas', 'Madeira', 'Solimoes']
df_sample = df[(df.Considerar == 1) & (df.Rio.isin(rios))]
df_sample = df_sample[['Rio', 'Nome', 'Conteineres/Ano']]

In [18]:
(35*50)/0.9

1944.4444444444443

In [19]:
400 / 50

8.0

In [20]:
# Parameters
vel_media = 10 * 1.852 # velocidade média do navio
t_carregamento = 4 * 5 / 60 # 5 minutos para cada movimento (4 movimentos por carga)
f_corredor = 0.9 # porcentagem da capacidade útil do navio
capac_navio_nominal = 2000 # capacidade nominal do navio em volume [m3]
capac_carga  = 50 # capacidade do conteiner em volume [m3]

# Frequencia de visitas específica por rio, ajustada pensando na capacidade do navio
freq_visitas = [2, 3, 2]

distancias_maximas_de_manaus = [458, 972.8, 2043]

In [21]:
# Capacidade do navio
capac_navio = (f_corredor * capac_navio_nominal) / capac_carga

for i in range(0, len(rios)):
    rio = rios[i]
    freq = freq_visitas[i]
    
    # Selecionar localidades "Consideradas" e de dado "Rio"
    df_sample_rio = df_sample[df_sample.Rio == rio]
    loc_rio = df_sample_rio['Nome'].tolist()
    # Maxima dist entre localidades (extremos)
    max_dist = distancias_maximas_de_manaus[i]

    # Tempo de Navegação é 2 vezes a distancia dividida pela vel 
    tempo_nav = 2 * max_dist / vel_media
    
    # Demanda total por ano (arredonda para cima)
    demanda_total = m.ceil(df_sample_rio['Conteineres/Ano'].sum())
    
    # Viagens redondas por ano (arredonda por cima)
    viagens_redondas = m.ceil(demanda_total / capac_navio)
    
    # Tempo de ciclo
    t_ciclo = (freq * tempo_nav + demanda_total/ (12 * 30 * 24) * t_carregamento)
    
    print(rio)
    print(f'Frequência de entrega: {freq} [-]')
    print(f'Distância máxima: {max_dist} [km]')
    print(f'Tempo de Navegação: {round(tempo_nav, 2)} [h]')
    print(f'Demanda total: {demanda_total} [cargas/ano]')
    print(f'Viagens redondas: {viagens_redondas} [viagens]')
    print(f'Tempo de ciclo: {round(t_ciclo, 2)} [h] ou {round(t_ciclo /24, 2)} [dias] \n')

Amazonas
Frequência de entrega: 2 [-]
Distância máxima: 458 [km]
Tempo de Navegação: 49.46 [h]
Demanda total: 4544 [cargas/ano]
Viagens redondas: 127 [viagens]
Tempo de ciclo: 99.1 [h] ou 4.13 [dias] 

Madeira
Frequência de entrega: 3 [-]
Distância máxima: 972.8 [km]
Tempo de Navegação: 105.05 [h]
Demanda total: 2493 [cargas/ano]
Viagens redondas: 70 [viagens]
Tempo de ciclo: 315.26 [h] ou 13.14 [dias] 

Solimoes
Frequência de entrega: 2 [-]
Distância máxima: 2043 [km]
Tempo de Navegação: 220.63 [h]
Demanda total: 3428 [cargas/ano]
Viagens redondas: 96 [viagens]
Tempo de ciclo: 441.38 [h] ou 18.39 [dias] 



Os números apresentados justificam a extensão do horizonte de planejamento para cada rio: No Amazonas, o tempo de ciclo de 4.54 dias indica um horizonte de 7 dias; no rio Madeira,  o tempo de ciclo de 10.3 dias indica um horizonte de 14 dias, e no Solimões 21 dias.

# Frequência de visitas

In [22]:
def _tempo_de_ciclo(data):
    if data == 'Amazonas':
        return 7
    if data == 'Madeira':
        return 14
    if data == 'Solimoes':
        return 21

In [23]:
df_sample['tempo_ciclo'] = df_sample.Rio.apply(_tempo_de_ciclo)

In [24]:
df_sample['demanda_por_ciclo'] = (df_sample['Conteineres/Ano'] / (12 * 30) * df_sample['tempo_ciclo'] ).apply(m.ceil)

# Simulando 1, 2, 3 ou 4 visitas por ciclo (para ver as aderencias e definir)
freqs_possiveis = [1, 2, 3, 4]
for f in freqs_possiveis:
    df_sample[f'demanda_{f}_visita'] = (df_sample['demanda_por_ciclo'] / f).apply(m.ceil)

df_sample[['Rio', 'Nome','demanda_1_visita', 'demanda_2_visita', 'demanda_3_visita', 'demanda_4_visita']]

,Rio,Nome,demanda_1_visita,demanda_2_visita,demanda_3_visita,demanda_4_visita
2,Solimoes,CAMPINAS,1,1,1,1
8,Solimoes,CAIAMBÉ,3,2,1,1
9,Solimoes,TEFÉ,60,30,20,15
10,Solimoes,ALVARÃES,8,4,3,2
11,Solimoes,UARINI,8,4,3,2
12,Solimoes,FONTE BOA,13,7,5,4
13,Solimoes,JUTAÍ,13,7,5,4
14,Solimoes,TONANTINS,7,4,3,2
15,Solimoes,SANTO ANTÔNIO DO IÇÁ,11,6,4,3
17,Solimoes,AMATURÁ,5,3,2,2


In [25]:
# Observando qual demanda por visita parece razoavel:
freq_visita = {'CAMPINAS': 1,
 'CAIAMBÉ': 1,
 'TEFÉ': 4,
 'ALVARÃES': 1,
 'UARINI': 1,
 'FONTE BOA': 2,
 'JUTAÍ': 2,
 'TONANTINS': 1,
 'SANTO ANTÔNIO DO IÇÁ': 1,
 'AMATURÁ': 1,
 'SÃO PAULO DE OLIVENÇA': 1,
 'SANTA RITA DO WEILL': 1,
 'BELÉM DO SOLIMÕES': 1,
 'FEIJOAL': 1,
 'BENJAMIN CONSTANT': 3,
 'TABATINGA': 4,
 'AUTAZES': 2,
 'NOVA OLINDA DO NORTE': 2,
 'AXINIM': 1,
 'BORBA': 2,
 'NOVO ARIPUANÃ': 1,
 'MANICORÉ': 2,
 'AUXILIADORA': 1,
 'HUMAITÁ': 3,
 'ITACOATIARA': 4,
 'NOVO REMANÇO': 1,
 'URUCURITUBA': 1,
 'AUGUSTO MONTENEGRO': 1,
 'URUCARÁ': 1,
 'PARINTINS': 3,
 'VILA AMAZÔNIA': 1
              }

In [26]:
def _demanda_por_visita(data):
    return data[data[4] - 1]

In [27]:
# Mapear frequencias observadas
df_sample['freq_visita'] = df_sample['Nome'].map(freq_visita)
df_sample['demanda_por_visita'] = df_sample.iloc[:, -5:].apply(_demanda_por_visita, axis=1)

In [28]:
df_sample[['Nome', 'Rio', 'demanda_por_visita', 'freq_visita']]

,Nome,Rio,demanda_por_visita,freq_visita
2,CAMPINAS,Solimoes,1,1
8,CAIAMBÉ,Solimoes,3,1
9,TEFÉ,Solimoes,15,4
10,ALVARÃES,Solimoes,8,1
11,UARINI,Solimoes,8,1
12,FONTE BOA,Solimoes,7,2
13,JUTAÍ,Solimoes,7,2
14,TONANTINS,Solimoes,7,1
15,SANTO ANTÔNIO DO IÇÁ,Solimoes,11,1
17,AMATURÁ,Solimoes,5,1


In [29]:
df_sample[['Nome', 'Rio', 'demanda_por_visita', 'freq_visita']]

,Nome,Rio,demanda_por_visita,freq_visita
2,CAMPINAS,Solimoes,1,1
8,CAIAMBÉ,Solimoes,3,1
9,TEFÉ,Solimoes,15,4
10,ALVARÃES,Solimoes,8,1
11,UARINI,Solimoes,8,1
12,FONTE BOA,Solimoes,7,2
13,JUTAÍ,Solimoes,7,2
14,TONANTINS,Solimoes,7,1
15,SANTO ANTÔNIO DO IÇÁ,Solimoes,11,1
17,AMATURÁ,Solimoes,5,1


In [30]:
cwd = os.getcwd()
pattern_path = os.path.join(cwd, '../data/preprocessing/departure_patterns/7_days.xlsx')
df_pattern_7_days = pd.read_excel(pattern_path)
pattern_path = os.path.join(cwd, '../data/preprocessing/departure_patterns/14_days.xlsx')
df_pattern_14_days = pd.read_excel(pattern_path)
pattern_path = os.path.join(cwd, '../data/preprocessing/departure_patterns/21_days.xlsx')
df_pattern_21_days = pd.read_excel(pattern_path)
pattern_path = os.path.join(cwd, '../data/preprocessing/departure_patterns/42_days.xlsx')
df_pattern_42_days = pd.read_excel(pattern_path)

def  _pattern_index(data, df_pattern):
    pattern_index = df_pattern[df_pattern.freq == data].num.tolist()
    return pattern_index

In [31]:
df_sample['pattern'] = 0
df_sample.loc[df_sample['Rio'] == 'Amazonas', 'pattern'] =  df_sample['freq_visita'].apply(_pattern_index, 
                                                                                           df_pattern=df_pattern_7_days)
df_sample.loc[df_sample['Rio'] == 'Madeira', 'pattern'] =  df_sample['freq_visita'].apply(_pattern_index, 
                                                                                           df_pattern=df_pattern_14_days)
df_sample.loc[df_sample['Rio'] == 'Solimoes', 'pattern'] =  df_sample['freq_visita'].apply(_pattern_index, 
                                                                                           df_pattern=df_pattern_21_days)

In [32]:
# Create dataframe above for each river, ordered by distance from Manaus
for rio in rios:
    lista_localidades = df_sample[df_sample.Rio == rio]['Nome'].tolist()
    localidades_ordenadas = df_dist.loc[lista_localidades].sort_values('MANAUS').index.tolist()
    sorterIndex = dict(zip(localidades_ordenadas, range(len(localidades_ordenadas))))
    
    df_freq_visita = df_sample.copy()
    df_freq_visita = df_freq_visita[df_sample.Rio == rio]
    df_freq_visita['sort'] = df_freq_visita['Nome'].map(sorterIndex)
    df_freq_visita = df_freq_visita.sort_values('sort')
    
    df_freq_visita = df_freq_visita[['Nome', 'Rio', 'demanda_por_visita', 'freq_visita', 'pattern']]
    df_freq_visita.to_parquet(f'../data/preprocessing/demands/{rio}.parquet')

### Modelo Multiplas viagens

In [36]:
rios_mult = ['Amazonas', 'Madeira']

df_sample['pattern'] = 0
df_sample.loc[df_sample['Rio'].isin(rios_mult), 'pattern'] =  df_sample['freq_visita'].apply(_pattern_index, 
                                                                                           df_pattern=df_pattern_14_days)

In [37]:
lista_localidades = df_sample[df_sample.Rio.isin(rios_mult)]['Nome'].tolist()
localidades_ordenadas = df_dist.loc[lista_localidades].sort_values('MANAUS').index.tolist()
sorterIndex = dict(zip(localidades_ordenadas, range(len(localidades_ordenadas))))

df_freq_visita = df_sample.copy()
df_freq_visita = df_freq_visita[df_sample.Rio.isin(rios_mult)]
df_freq_visita['sort'] = df_freq_visita['Nome'].map(sorterIndex)
df_freq_visita = df_freq_visita.sort_values(['Rio', 'sort'])

df_freq_visita = df_freq_visita[['Nome', 'Rio', 'demanda_por_visita', 'freq_visita', 'pattern']]
df_freq_visita.to_parquet(f'../data/preprocessing/demands/{rios_mult}.parquet')

In [38]:
df_freq_visita

,Nome,Rio,demanda_por_visita,freq_visita,pattern
76,NOVO REMANÇO,Amazonas,7,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
75,ITACOATIARA,Amazonas,11,4,"[35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 4..."
77,URUCURITUBA,Amazonas,5,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
78,AUGUSTO MONTENEGRO,Amazonas,1,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
79,URUCARÁ,Amazonas,4,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
80,PARINTINS,Amazonas,10,3,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 3..."
82,VILA AMAZÔNIA,Amazonas,2,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
55,AUTAZES,Madeira,8,2,"[14, 15, 16, 17, 18, 19, 20]"
56,NOVA OLINDA DO NORTE,Madeira,7,2,"[14, 15, 16, 17, 18, 19, 20]"
57,AXINIM,Madeira,1,1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"


# Geração de Rotas

- Gerar todas as combinações de localidades (contendo 1, 2, 3, 4, etc.) para cada rio, até que a capacidade do navio se esgote (Ter as localidades ordenadas em ordem crescente de distância a Manaus).
- A cada rota, registrar:
    - tempo de ciclo da rota (separado por tempo navegando e tempo de porto)
    - demanda total da rota,
    - custo de navegação da rota
    - se a localidade i está na rota r ou não

## Loop principal

In [30]:
# Capacidade maxima da embarcacao
cap_max = 35 # 2000 * 0.9 / 50 = 36
vel_media = 10 * 1.852 # velocidade média do navio
t_carregamento = 4 * 5 / 60 # 5 minutos para cada movimento (4 movimentos por carga)

In [31]:
def vetor_no_pertence_a_rota(data, localidades_ordenadas):
    vetor = np.zeros(len(localidades_ordenadas))
    for i in range(0, len(vetor)):
        if localidades_ordenadas[i] in data:
            vetor[i] = 1
    return vetor

In [42]:
def make_route(data: pd.DataFrame, 
               matrix_distancias: pd.DataFrame, 
               rio: str) -> pd.DataFrame:
    
    # Ordenar localidades por distancia a Manaus
    lista_localidades = data[data.Rio == rio]['Nome'].tolist()
    localidades_ordenadas = matrix_distancias.loc[lista_localidades].sort_values('MANAUS').index.tolist()
    
    localidades =  data.set_index('Nome').loc[localidades_ordenadas].index.tolist()
    demandas = data.set_index('Nome').loc[localidades_ordenadas]['demanda_por_visita'].tolist()
    
    rotas = []
    demandas_rotas = []

    distancias_manaus = []
    tempos_porto = []
    tempos_nav = []
    tempos_ciclo = []

    num_rotas_totais = 0

    for i in range(1, len(localidades) + 1): # Para cada combinacao (1 a 1, 2 a 2....)
     #   print(f'Combinações {i} a {i}:')
        comb_demandas = list(combinations(demandas,i))
        comb_locais = list(combinations(localidades,i))

        target_capac_max = 0
        for j, demanda_comb in enumerate(comb_demandas): # Para cada rota nas combinacoes
            demanda_comb = sum(demanda_comb)
            if demanda_comb <= cap_max: # se atingir a capacidade maxiam do navio
    #            print(f'Demanda total da rota: {demanda_comb}')
                demandas_rotas.append(demanda_comb)

                target_capac_max +=1
                # Estabelecer os nos da combinacao
                nos_comb = list(comb_locais[j])
    #            print(f'Rota: {nos_comb}')
                rotas.append(nos_comb)

                dist_manaus = 0
                dist_manaus += df_dist.loc[nos_comb[0], 'MANAUS']
                final = df_dist.loc[nos_comb[-1], 'MANAUS']
  #              print(f'Distância do Primeiro local até Manaus: {dist_manaus}')
  #              print(f'Distância último local até Manaus: {final}')
                for k in range(0, len(nos_comb)):
                    if k < len(nos_comb) - 1:
                        no_anterior = nos_comb[k]
                        no_posterior = nos_comb[k+1]
                        distancia_entre_nos = df_dist.loc[no_anterior, no_posterior]
    #                    print(f'Arcos: {no_anterior} + {no_posterior}')
   #                     print(f'Distância do arco: {distancia_entre_nos}')
                        dist_manaus += distancia_entre_nos

                dist_manaus = round(dist_manaus, 2)
    #            print(f'Distância percorrida: {round(dist_manaus,2)}')
                distancias_manaus.append(dist_manaus)

                tempo_porto = round(demanda_comb * t_carregamento, 2)
                tempo_nav = round(2 * dist_manaus / vel_media, 2)
                tempo_ciclo = round(tempo_porto + tempo_nav, 2)

    #            print(f'Tempo de porto: {tempo_porto}')
                tempos_porto.append(tempo_porto)
    #            print(f'Tempo de nav: {tempo_nav}')
                tempos_nav.append(tempo_nav)
    #            print(f'Tempo de ciclo: {tempo_ciclo}')
                tempos_ciclo.append(tempo_ciclo)

    #            print('*******************************************')
    #    print(f'Total de rotas com {i} localidades: {target_capac_max}')
        num_rotas_totais += target_capac_max
    #    print('-------------------------------------------------')

        if target_capac_max == 0:
    #        print('Não há combinações maiores possíveis\n')
            break
    
    #print(f'Número total de rotas: {num_rotas_totais}\n')
    #print(f'Lista de rotas: {rotas}\n')
    #print(f'Demanda [containers] das rotas: {demandas_rotas}\n')
    #print(f'Tempo [h] de porto das rotas: {tempos_porto}')
    #print(f'Tempo [h] de navegação nas rotas: {tempos_nav}')
    #print(f'Tempo [h] de ciclo nas rotas: {tempos_ciclo}')
    #print(f'Distâncias máximas até Manaus: {distancias_manaus}')
    
    dict_to_dataframe = {
    'nos': pd.Series(rotas),
    'demanda': pd.Series(demandas_rotas),
    'tempo_porto': pd.Series(tempos_porto),
    'tempo_nav': pd.Series(tempos_nav),
    'tempo_ciclo': pd.Series(tempos_ciclo),
    'distancia': pd.Series(distancias_manaus),
}
    
    rotas = pd.DataFrame.from_dict(dict_to_dataframe, orient='index').T
    rotas['vetor_loc_rota'] = rotas['nos'].apply(vetor_no_pertence_a_rota, localidades_ordenadas =localidades_ordenadas)
    
    return rotas

In [47]:
# Escolha o rio que quer gerar as rotas
rio = 'Amazonas'  # 'Amazonas', 'Madeira', 'Solimões'

rota = make_route(data=df_sample, 
                  matrix_distancias=df_dist,
                  rio=rio)

In [48]:
# Salvar a base
folderpath_rotas = '../data/preprocessing/routes/'
rota.to_parquet(folderpath_rotas + f'rotas_{rio}.parquet')